In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import ImageTk, Image, ImageDraw
import PIL
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time

# Create data directory if it doesn't exist
try:
    os.mkdir('data')
except:
    print('Path Already Exists')

# GUI dimensions
width, height = 600, 600
padding = 20  # padding around elements

# Initialize main window
win = tk.Tk()
win.title("Sinhala Character Recognition")
win.configure(bg="#f0f0f0")
win.resizable(False, False)

# Center window on screen
screen_width = win.winfo_screenwidth()
screen_height = win.winfo_screenheight()
x = (screen_width - width) // 2
y = (screen_height - height) // 2
win.geometry(f"{width + 2*padding}x{height + 150}+{x}+{y}")

# Fonts
font_btn = ('Helvetica', 14, 'bold')
font_label = ('Helvetica', 18, 'bold')
font_info = ('Helvetica', 12)
count = 0

# Load the pre-trained model
import joblib
clsfr = joblib.load('sinhala-character-knn.sav')

# Dictionary to map prediction index to Sinhala character
label_dict = {0:'අ', 1:'එ', 2:'ඉ', 3:'උ'}
char_names = {0:'a', 1:'ae', 2:'e', 3:'u'}

# Function to handle drawing events
def event_function(event):
    x = event.x
    y = event.y
    
    # Size of the brush
    brush_size = 15
    x1 = x - brush_size
    y1 = y - brush_size
    x2 = x + brush_size
    y2 = y + brush_size
    
    # Draw on canvas and in memory
    canvas.create_oval((x1, y1, x2, y2), fill='black', outline='black')
    img_draw.ellipse((x1, y1, x2, y2), fill='black', outline='black')

# Function to save the drawn character
def save():
    global count
    
    # Provide feedback
    status_var.set("Saving...")
    win.update()
    
    # Process image
    img_array = np.array(img)
    img_array = cv2.resize(img_array, (8, 8))
    
    # Save image
    path = os.path.join('data', str(count) + '.jpg')
    cv2.imwrite(path, img_array)
    
    # Update counter and display feedback
    count = count + 1
    status_var.set(f"Image saved as {count-1}.jpg")
    messagebox.showinfo("Image Saved", f"Character saved to data/{count-1}.jpg")

# Function to clear the canvas
def clear():
    global img, img_draw
    
    # Clear canvas and memory
    canvas.delete('all')
    img = Image.new('RGB', (width, height), (255, 255, 255))
    img_draw = ImageDraw.Draw(img)
    
    # Reset status
    prediction_var.set("NONE")
    status_var.set("Ready to draw")
    confidence_var.set("")
    
    # Redraw canvas border
    canvas.create_rectangle(2, 2, width-2, height-2, outline="#cccccc", width=2)

# Function to predict the drawn character
def predict():
    # Update status
    status_var.set("Predicting...")
    win.update()
    
    # Process image for prediction
    img_array = np.array(img)
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    img_array = cv2.resize(img_array, (8, 8))
    
    # Optional: show the processed image
    # plt.figure(figsize=(3,3))
    # plt.imshow(img_array, cmap='binary')
    # plt.title("Processed Image (8x8)")
    # plt.show()
    
    # Reshape for prediction
    img_array = np.reshape(img_array, (1, 64))
    
    # Make prediction with confidence scores
    result = clsfr.predict(img_array)[0]
    
    # Get probabilities if available
    try:
        probs = clsfr.predict_proba(img_array)[0]
        confidence = probs[result] * 100  # Convert to percentage
        confidence_text = f"Confidence: {confidence:.1f}%"
    except:
        confidence_text = ""
    
    # Update prediction display
    label = label_dict[result]
    char_name = char_names[result]
    prediction_var.set(f"{label} ({char_name})")
    confidence_var.set(confidence_text)
    status_var.set("Prediction complete")

# Create main frame
main_frame = tk.Frame(win, bg="#f0f0f0", padx=padding, pady=padding)
main_frame.pack(fill="both", expand=True)

# Create canvas with border
canvas_frame = tk.Frame(main_frame, bg="#f0f0f0", padx=0, pady=0)
canvas_frame.pack(padx=10, pady=10)

canvas = tk.Canvas(canvas_frame, width=width, height=height, bg='white', highlightthickness=0)
canvas.pack()
canvas.create_rectangle(2, 2, width-2, height-2, outline="#cccccc", width=2)

# Create control frame for buttons
control_frame = tk.Frame(main_frame, bg="#f0f0f0", padx=10, pady=10)
control_frame.pack(fill="x")

# Create fancy buttons with hover effect
class HoverButton(tk.Button):
    def __init__(self, master, **kw):
        tk.Button.__init__(self, master=master, **kw)
        self.defaultBackground = self["background"]
        self.defaultForeground = self["foreground"]
        self.bind("<Enter>", self.on_enter)
        self.bind("<Leave>", self.on_leave)

    def on_enter(self, e):
        darker_bg = self.rgb_to_darker(self["background"])
        self["background"] = darker_bg

    def on_leave(self, e):
        self["background"] = self.defaultBackground
        
    def rgb_to_darker(self, color):
        # Simple function to darken a color
        if color.startswith('#'):
            # Convert hex to RGB
            r = int(color[1:3], 16)
            g = int(color[3:5], 16)
            b = int(color[5:7], 16)
            
            # Darken by 20%
            factor = 0.8
            r = int(r * factor)
            g = int(g * factor)
            b = int(b * factor)
            
            return f"#{r:02x}{g:02x}{b:02x}"
        return color

# Create buttons with consistent styling
button_save = HoverButton(control_frame, text='SAVE', bg="#4CAF50", fg="white", font=font_btn, 
                     command=save, width=10, relief=tk.RAISED, borderwidth=2)
button_save.grid(row=0, column=0, padx=5, pady=5)

button_predict = HoverButton(control_frame, text='PREDICT', bg="#2196F3", fg="white", font=font_btn,
                        command=predict, width=10, relief=tk.RAISED, borderwidth=2)
button_predict.grid(row=0, column=1, padx=5, pady=5)

button_clear = HoverButton(control_frame, text='CLEAR', bg="#FFC107", fg="white", font=font_btn,
                      command=clear, width=10, relief=tk.RAISED, borderwidth=2)
button_clear.grid(row=0, column=2, padx=5, pady=5)

button_exit = HoverButton(control_frame, text='EXIT', bg="#F44336", fg="white", font=font_btn,
                     command=win.destroy, width=10, relief=tk.RAISED, borderwidth=2)
button_exit.grid(row=0, column=3, padx=5, pady=5)

# Status frame for prediction display
status_frame = tk.Frame(main_frame, bg="#f0f0f0", padx=10, pady=5)
status_frame.pack(fill="x")

# Create variables for dynamic text
prediction_var = tk.StringVar(value="NONE")
confidence_var = tk.StringVar(value="")
status_var = tk.StringVar(value="Ready to draw")

# Prediction display
prediction_label = tk.Label(status_frame, text="PREDICTED CHARACTER:", bg="#f0f0f0", font=font_info)
prediction_label.grid(row=0, column=0, sticky="w")

prediction_value = tk.Label(status_frame, textvariable=prediction_var, bg="#f0f0f0", font=font_label, fg="#0D47A1")
prediction_value.grid(row=0, column=1, padx=10)

confidence_label = tk.Label(status_frame, textvariable=confidence_var, bg="#f0f0f0", font=font_info, fg="#666666")
confidence_label.grid(row=0, column=2, padx=10)

# Status bar at bottom
status_bar = tk.Label(win, textvariable=status_var, bd=1, relief=tk.SUNKEN, anchor=tk.W, bg="#e0e0e0")
status_bar.pack(side=tk.BOTTOM, fill=tk.X)

# Available characters display
info_frame = tk.Frame(main_frame, bg="#f0f0f0", padx=10, pady=5)
info_frame.pack(fill="x")

info_text = "Available characters: අ (a), එ (ae), ඉ (e), උ (u)"
info_label = tk.Label(info_frame, text=info_text, bg="#f0f0f0", font=font_info, fg="#666666")
info_label.pack()

# Bind drawing events
canvas.bind('<B1-Motion>', event_function)

# Initialize drawing variables
img = Image.new('RGB', (width, height), (255, 255, 255))
img_draw = ImageDraw.Draw(img)

# Start the app
win.mainloop()

Path Already Exists
